In [41]:
from ga4stpg.graph import UGraph

In [42]:
from collections import deque

In [43]:
edges = [
    (1, 3),
    (1, 15),
    (15, 17),
    (3, 5),
    (5, 11),
    (11, 13),
    (5, 7),
    (7, 9)
]

graph = UGraph()

for v, u in edges:
    graph.add_edge(v, u)

In [44]:
def find_segments(graph, common_nodes, start):
    visited = set()
    stack_outter = [start]
    result = list()

    def search(start, neighbor):
        segment = set()
        segment.add( (start, neighbor) )
        visited.add(start)
        stack_inner = [neighbor]

        while stack_inner:
            u = stack_inner.pop()
            visited.add(u)
            if u not in common_nodes:
                for w in graph.adjacent_to(u):
                    if w not in visited:
                        stack_inner.append(w)
                        segment.add((u, w))
            else:
                stack_outter.append(u)
        # end while
        return segment
        # end search

    while stack_outter:
        s = stack_outter.pop()
        visited.add(s)
        for v in graph.adjacent_to(s):
            if v not in visited:
                seg = search(s, v) 
                result.append(seg)

    return result

In [45]:
common_nodes = {1, 17, 5, 13, 9}
start = 1

In [46]:
find_segments(graph, common_nodes, start)

[{(1, 3), (3, 5)}, {(1, 15), (15, 17)}, {(5, 11), (11, 13)}, {(5, 7), (7, 9)}]

## Segundo Exemplo

In [47]:
edges = [
    (1, 3),
    (1, 15),
    (15, 17),
    (3, 5),
    (5, 11),
    (11, 13),
    (5, 7),
    (7, 9),
    (7, 23)
]

graph = UGraph()

for v, u in edges:
    graph.add_edge(v, u)

In [48]:
common_nodes = {1, 17, 5, 13, 9, 23}
start = 17

seg = find_segments(graph, common_nodes, start)

for s in seg:
    print(s)

{(15, 1), (17, 15)}
{(1, 3), (3, 5)}
{(11, 13), (5, 11)}
{(5, 7), (7, 23), (7, 9)}


## Identificando partições para cruzamento de duas soluções

In [49]:
from ga4stpg.edgeset import UEdge, EdgeSet

class Segment:

    def __init__(self):
        self.edges = EdgeSet()
        self.cost = 0
        self.portals = set()

    def __len__(self):
        return len(self.edges)

    def __str__(self):
        return f'Segment <{len(self.edges)}>'

    def __iter__(self):
        return iter(self.edges)

    @property
    def bounds(self):
        return frozenset(self.portals)
    
    def add(self, v, u):
        self.edges.add(UEdge(v, u))
    

In [50]:
def f_weight(v, u):
    if STPG.graph.has_edge(v, u):
        return STPG.graph.weight(v, u)
    else:
        return 0

def find_segments(graph, common_nodes):
    visited = set()
    stack_outter = list(common_nodes)
    result = list()

    def search(start, neighbor):
        segment = Segment()
        segment.add(start, neighbor)
        segment.cost += f_weight(start, neighbor)
        segment.portals.add(start)
        
        visited.add(start)
        stack_inner = [neighbor]

        while stack_inner:
            u = stack_inner.pop()
            visited.add(u)
            if u not in common_nodes:
                for w in graph.adjacent_to(u):
                    if w not in visited:
                        stack_inner.append(w)
                        segment.add(u, w)
                        segment.cost += f_weight(u, w)
            else:
                stack_outter.append(u)
                segment.portals.add(u)
        # end while
        return segment
        # end search

    while stack_outter:
        s = stack_outter.pop()
        visited.add(s)
        for v in graph.adjacent_to(s):
            if v not in visited:
                seg = search(s, v) 
                result.append(seg)

    return result

In [51]:
from ga4stpg.graph.reader import read_problem
from ga4stpg.util import STEIN_B

from ga4stpg.tree.evaluation import EvaluateTreeGraph
from ga4stpg.tree.generate import GenerateBasedPrimRST
from ga4stpg.tree.mutate import ReplaceByRandomEdge
from ga4stpg.tree.pxcrossover import PXTree

In [52]:
from os import path

In [53]:
STEIN_B

[('steinb1.txt', 82),
 ('steinb2.txt', 83),
 ('steinb3.txt', 138),
 ('steinb4.txt', 59),
 ('steinb5.txt', 61),
 ('steinb6.txt', 122),
 ('steinb7.txt', 111),
 ('steinb8.txt', 104),
 ('steinb9.txt', 220),
 ('steinb10.txt', 86),
 ('steinb11.txt', 88),
 ('steinb12.txt', 174),
 ('steinb13.txt', 165),
 ('steinb14.txt', 235),
 ('steinb15.txt', 318),
 ('steinb16.txt', 127),
 ('steinb17.txt', 131),
 ('steinb18.txt', 218)]

In [54]:
INDEX_SELECT = 4

datasets = path.join('..', '..', 'ppgi-stpg-gpx', 'datasets', 'ORLibrary')

filename = path.join(datasets, STEIN_B[INDEX_SELECT][0])

print(filename)

STPG = read_problem(filename)

..\..\ppgi-stpg-gpx\datasets\ORLibrary\steinb5.txt


In [55]:
STPG.name

'B5'

In [56]:
generator = GenerateBasedPrimRST(STPG)

red  = generator()
blue = generator()

In [57]:
red.edges == blue.edges

False

In [58]:
child     = UGraph()
red_only  = UGraph()
blue_only = UGraph()

In [59]:
for v, u in red.gen_undirect_edges():
    if blue.has_edge(v, u):
        child.add_edge(v, u)
    else:
        red_only.add_edge(v, u)

In [60]:
for v, u in blue.gen_undirect_edges():
    if not red.has_edge(v, u):
        blue_only.add_edge(v, u)

In [61]:
common_nodes = set(red_only.vertices) & set(blue_only.vertices)
terminals = STPG.terminals.copy()

common_nodes & terminals

{3, 5, 7, 15, 20, 23, 24, 37}

In [62]:
len(common_nodes)

25

In [63]:
red_segments  = find_segments(red_only, common_nodes)
blue_segments = find_segments(blue_only, common_nodes)

len(red_segments)

18

In [64]:
len(blue_segments)

19

In [65]:
from ga4stpg.graph.priorityqueue import PriorityQueue

queue = PriorityQueue()

In [66]:
for seg in red_segments:
    print(seg, seg.cost)
    queue.push(seg.cost, seg)

Segment <1> 1
Segment <1> 2
Segment <1> 5
Segment <1> 4
Segment <3> 8
Segment <1> 8
Segment <2> 15
Segment <1> 1
Segment <1> 3
Segment <1> 1
Segment <1> 5
Segment <1> 4
Segment <1> 8
Segment <2> 14
Segment <1> 2
Segment <1> 1
Segment <1> 1
Segment <1> 3


In [67]:
for seg in blue_segments:
    print(seg, seg.cost)
    queue.push(seg.cost, seg)

Segment <1> 2
Segment <1> 10
Segment <1> 7
Segment <5> 28
Segment <1> 6
Segment <1> 7
Segment <1> 5
Segment <1> 6
Segment <1> 9
Segment <1> 5
Segment <1> 4
Segment <1> 4
Segment <1> 5
Segment <1> 5
Segment <1> 10
Segment <1> 2
Segment <1> 7
Segment <1> 7
Segment <1> 2


In [68]:
len(queue)

37

In [69]:
from ga4stpg.graph.util import has_cycle, how_many_components

In [70]:
has_cycle(child)

False

In [71]:
how_many_components(child)

12

In [72]:
from ga4stpg.graph.disjointsets import DisjointSets

In [73]:
dsets = DisjointSets()

for v in child.vertices:
    dsets.make_set(v)

for v, u in child.gen_undirect_edges():
    dsets.union(v, u)

In [74]:
for v in common_nodes:
    if v not in dsets:
        dsets.make_set(v)

In [75]:
while queue:
    seg = queue.pop()
    test = set()
    for k in seg.portals:
        test.add(dsets.find(k))
    if len(test) == len(seg.portals):
        print('adding segment', seg)
        for edge in seg:
            v, u = edge 
            child.add_edge(v, u)
        portals = list(seg.portals)
        for p1, p2 in zip(portals[0::2], portals[1::2]):
            dsets.union(p1, p2)
    else:
        print("####### >>> rejecting: ", seg)

adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
####### >>> rejecting:  Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
adding segment Segment <1>
####### >>> rejecting:  Segment <3>
adding segment Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
####### >>> rejecting:  Segment <1>
adding segment Segment <1>
adding segment Segm

In [76]:
len(queue)

0

In [77]:
beta = '010101010101010101'

for a, b in zip(beta[0::2],beta[1::2]):
    # print(b)
    pass

In [78]:
has_cycle(child)

True

In [79]:
how_many_components(child)

1

In [80]:
child.edges

defaultdict(set,
            {25: {5, 14, 15, 43},
             43: {23, 25},
             5: {18, 25, 48},
             48: {5, 26, 27},
             18: {1, 5, 6, 10},
             1: {7, 18, 35},
             35: {1},
             26: {6, 48},
             6: {12, 18, 21, 26, 50},
             12: {6, 15},
             50: {6},
             21: {3, 6, 9, 16, 17, 37},
             3: {20, 21, 39},
             30: {32},
             32: {11, 30, 36, 42},
             39: {3},
             37: {8, 21, 34, 40},
             40: {37},
             10: {13, 18, 19},
             19: {10},
             13: {10},
             42: {32, 45},
             36: {8, 24, 32},
             23: {31, 33, 34, 43},
             33: {23},
             31: {23},
             2: {16},
             16: {2, 21},
             46: {7, 38, 47},
             38: {46},
             45: {8, 42},
             47: {20, 22, 46},
             22: {47},
             4: {24, 28, 41},
             41: {4},
            